In [1]:
# Setup CustomVision credentials

from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

endpoint = 'PREDICTION ENDPOINT' 
prediction_key = 'PREDICTION KEY' 
project_id = 'PROJECT ID' 
iteration_id = 'ITERATION ID'
predictor = CustomVisionPredictionClient(prediction_key, endpoint=endpoint)

In [2]:
# Load prerequisits 

import mPyPl as mp
import json 
from os.path import join
from metrics import *
from PIL import Image

data_dir = 'data/images'
vott_export = json.load(open('data/images.json', 'r'))
input_tags = vott_export['inputTags'].split(',')

In [3]:
# Define some useful functions 

def ignore_result(func, value):
    return func or value

def box_to_whformat(box):
    box['width'] = abs(box['x2'] - box['x1'])
    box['height'] = abs(box['y2'] - box['y1'])
    box.pop('x2', None)
    box.pop('y2', None)
    return box

def prediction_as_dict(prediction, width=None, height=None):
    return {
        'tag' : prediction.tag_name,
        'prob' : prediction.probability,
        'x1' : prediction.bounding_box.left * width if width else prediction.bounding_box.left,
        'y1' : prediction.bounding_box.top * height if height else prediction.bounding_box.top,
        'width' : prediction.bounding_box.width * width if width else prediction.bounding_box.width,
        'height' : prediction.bounding_box.height * height if height else prediction.bounding_box.height
    }

def format_dict(prediction, width=None, height=None):
    return {
        'tag' : prediction['tagName'],
        'prob' : prediction['probability'],
        'x1' : prediction['boundingBox']['left'] * width if width else prediction['boundingBox']['left'],
        'y1' : prediction['boundingBox']['top'] * height if height else prediction['boundingBox']['top'],
        'width' : prediction['boundingBox']['width'] * width if width else prediction['boundingBox']['width'],
        'height' : prediction['boundingBox']['height'] * height if height else prediction['boundingBox']['height']
    }

def print_report(stream, input_tags, pred_field='predictions', gt_field='ground_truth'):
    top = '{:15.12} | {:^12.10} | {:^12.10}'.format("Tag", "Precision", "Recall")
    print(top)
    print('-' * len(top))
    for tag in input_tags:
        precision, recall = precision_recall(stream, tag, 0.35, 0.3, pred_field=pred_field, gt_field=gt_field)
        print('{:15.12} | {:^12.5} | {:^12.5}'.format(tag, float(precision), float(recall)))

In [7]:
# Compress CoreML model

import coremltools

# Option 1: convert a full precision (float) MLModel to a 16bit quantized MLModel
model_spec = coremltools.utils.load_spec('models/coreml/model.mlmodel')
model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
coremltools.utils.save_spec(model_fp16_spec, 'models/coreml/modelFP16.mlmodel')

# Option 2: convert a full precision (float) MLModel to a 8bit quantized MLModel
model = coremltools.models.MLModel('models/coreml/model.mlmodel')
model_fp8 = coremltools.models.neural_network.quantization_utils.quantize_weights(model, nbits=8)
model_fp8.save('models/coreml/modelFP8.mlmodel')

Quantizing using linear quantization
Optimizing Neural Network before Quantization:
Finished optimizing network. Quantizing neural network..
Quantizing layer layer1/conv
Quantizing layer layer2/conv
Quantizing layer layer3/conv
Quantizing layer layer4/conv
Quantizing layer layer5/conv
Quantizing layer layer6/conv
Quantizing layer layer7/conv
Quantizing layer layer8/conv
Quantizing layer model_outputs0


In [5]:
# Setup CoreML models

from models.coreml.python.predict import *

coreml_16 = coremltools.models.MLModel('models/coreml/modelFP16.mlmodel')
coreml_8 = coremltools.models.MLModel('models/coreml/modelFP8.mlmodel')
coreml_orig = coremltools.models.MLModel('models/coreml/model.mlmodel')

with open('models/coreml/labels.txt', 'r') as f:
    labels = [l.strip() for l in f.readlines()]

od_coreml_16 = CoreMLObjectDetection(coreml_16, labels)
od_coreml_8 = CoreMLObjectDetection(coreml_8, labels)
od_coreml_orig = CoreMLObjectDetection(coreml_orig, labels)

In [6]:
# Compress TensorFlow model
import tensorflow as tf

converter = tf.contrib.lite.TFLiteConverter.from_frozen_graph('models/tensorflow/model.pb', ['Placeholder'], ['model_outputs'])
converter.post_training_quantize = True
tflite_model_8 = converter.convert()
open("models/tensorflow/modelFP8.tflite", "wb").write(tflite_model_8)

11047480

In [8]:
# Setup TensorFlow Original Model

from models.tensorflow.python.predict import TFObjectDetection as OrigTFObjectDetection
from models.tensorflow.python.predict_lite import TFObjectDetection as LiteTFObjectDetection

graph_orig = tf.GraphDef()
with tf.gfile.FastGFile('models/tensorflow/model.pb', 'rb') as f:
    graph_orig.ParseFromString(f.read())
        
# Load labels
with open('models/tensorflow/labels.txt', 'r') as f:
    labels = [l.strip() for l in f.readlines()]

od_tflite_orig = OrigTFObjectDetection(graph_orig, labels)

od_tflite_8 = LiteTFObjectDetection('models/tensorflow/modelFP8.tflite', labels)

Instructions for updating:
Use tf.gfile.GFile.


In [9]:
# Setup datastream

stream = (
    list(vott_export['frames'].keys()) 
    | mp.as_field('filename')
    | mp.apply('filename', 'meta', lambda x: vott_export['frames'][x])
    | mp.filter('meta', lambda x: len(x) > 0)
    | mp.apply('meta', 'width', lambda x: x[0]['width'])
    | mp.apply('meta', 'height', lambda x: x[0]['height'])
    | mp.apply('meta', 'ground_truth', lambda x: x 
        | mp.select(lambda m: ignore_result(m['box'].update({'tag' : m['tags'][0]}), m['box'])) 
        | mp.select(lambda m: box_to_whformat(m))
        | mp.as_list
     )
    # Predict using online CustomVision
    | mp.apply('filename', 'raw_cv', 
        lambda x: predictor.predict_image(project_id, open(join(data_dir, x), mode="rb"), iteration_id).predictions)
    | mp.apply(['raw_cv', 'width', 'height'], 'cv_predictions', lambda x: x[0]
        | mp.select(lambda p: prediction_as_dict(p, x[1], x[2]))
        | mp.as_list
      )    
    # Predict using CoreML original
    | mp.apply('filename', 'coreml_orig_raw', lambda x: od_coreml_orig.predict_image(Image.open(join(data_dir, x))))
    | mp.apply(['coreml_orig_raw', 'width', 'height'], 'coreml_orig_predictions', lambda x: x[0]
        | mp.select(lambda p: format_dict(p, x[1], x[2]))
        | mp.as_list
      )  
    # Predict using CoreML compressed to 16FP
    | mp.apply('filename', 'coreml_16_raw', lambda x: od_coreml_16.predict_image(Image.open(join(data_dir, x))))
    | mp.apply(['coreml_16_raw', 'width', 'height'], 'coreml_16_predictions', lambda x: x[0]
        | mp.select(lambda p: format_dict(p, x[1], x[2]))
        | mp.as_list
      )  
    # Predict using CoreML compressed to 8FP
    | mp.apply('filename', 'coreml_8_raw', lambda x: od_coreml_8.predict_image(Image.open(join(data_dir, x))))
    | mp.apply(['coreml_8_raw', 'width', 'height'], 'coreml_8_predictions', lambda x: x[0]
        | mp.select(lambda p: format_dict(p, x[1], x[2]))
        | mp.as_list
      )  
    # Predict using TensorFlow original
    | mp.apply('filename', 'tf_orig_raw', lambda x: od_tflite_orig.predict_image(Image.open(join(data_dir, x))))
    | mp.apply(['tf_orig_raw', 'width', 'height'], 'tf_orig_predictions', lambda x: x[0]
        | mp.select(lambda p: format_dict(p, x[1], x[2]))
        | mp.as_list
      ) 
    # Predict using TensorFlow compressed to 8FP
    | mp.apply('filename', 'tf_8_raw', lambda x: od_tflite_8.predict_image(Image.open(join(data_dir, x))))
    | mp.apply(['tf_8_raw', 'width', 'height'], 'tf_8_predictions', lambda x: x[0]
        | mp.select(lambda p: format_dict(p, x[1], x[2]))
        | mp.as_list
      ) 
    | mp.delfield(['meta', 'raw_cv', 'coreml_orig_raw', 'coreml_16_raw', 'coreml_8_raw', 'tf_orig_raw', 'tf_8_raw'])
    | mp.as_list
)

In [10]:
stream[0]

{'filename': '000000017029.jpg',
 'width': 640,
 'height': 640,
 'ground_truth': [{'x1': 152.34454433313397,
   'y1': 136.70542155726676,
   'tag': 'dog',
   'width': 253.72514204545456,
   'height': 400.71093983627395}],
 'cv_predictions': [{'tag': 'cat',
   'prob': 0.0251565184,
   'x1': 1.6910696,
   'y1': 32.016565824,
   'width': 22.243554624,
   'height': 69.92457408},
  {'tag': 'cat',
   'prob': 0.0114600845,
   'x1': 166.8371008,
   'y1': 43.881788224,
   'width': 33.343028992,
   'height': 29.4543936},
  {'tag': 'cat',
   'prob': 0.0176783111,
   'x1': 1.3704752895999999,
   'y1': 37.157950400000004,
   'width': 50.3039552,
   'height': 63.8039808},
  {'tag': 'cat',
   'prob': 0.01035777,
   'x1': 147.9061696,
   'y1': 25.294544704000003,
   'width': 81.60093312000001,
   'height': 74.20054464},
  {'tag': 'cat',
   'prob': 0.0148153463,
   'x1': 4.5161819456000005,
   'y1': 451.24053951999997,
   'width': 199.10659776,
   'height': 83.392256},
  {'tag': 'dog',
   'prob': 0.077

In [11]:
import pickle

pickle.dump(stream, open('results.pickle', 'wb'))
# stream = pickle.load(open('results.pickle', 'rb'))

In [12]:
# Print reports

for pred_field in [key for key in stream[0].keys() if 'predictions' in key]:
    print("\n===== %s =====\n" % pred_field)
    print_report(stream, input_tags, pred_field=pred_field, gt_field='ground_truth')


===== cv_predictions =====

Tag             |  Precision   |    Recall   
---------------------------------------------
cat             |   0.89474    |   0.15741   
dog             |     0.5      |   0.23894   

===== coreml_orig_predictions =====

Tag             |  Precision   |    Recall   
---------------------------------------------
cat             |     1.0      |   0.083333  
dog             |   0.55556    |   0.26549   

===== coreml_16_predictions =====

Tag             |  Precision   |    Recall   
---------------------------------------------
cat             |     1.0      |   0.083333  
dog             |   0.55556    |   0.26549   

===== coreml_8_predictions =====

Tag             |  Precision   |    Recall   
---------------------------------------------
cat             |   0.31579    |   0.11111   
dog             |     0.52     |   0.23009   

===== tf_orig_predictions =====

Tag             |  Precision   |    Recall   
---------------------------------------------
